## Task 1: sampling. 
### First of all, let's load the data.  The Honeypot data. The total package is 34GB with 47 seperate csv files. We will load the first file and take a look.

In [1]:
import pandas as pd
# honeypot = pd.read_csv('two_month/cmds_sequence_2016-07-01.csv',sep = '$$')

### Next, we import the pre-processed data with only source IP addresses for the entire period. 
Data downloaded from slack, made by Qin lin.

In [2]:
colname = ['IP']
sourceIP = pd.read_csv('src_ip.txt',header=None,names = colname)
# sourceIP.columns =  'IP'
sourceIP[ :5]

,IP
0,115.177.11.215
1,220.134.245.30
2,218.77.202.162
3,14.145.251.40
4,221.154.68.198


In [3]:
# Let's see how many IP records do we have?
print('There are {} records in total.'.format(len(sourceIP)))

There are 3414011 records in total.


In [4]:
## Let's see how many unique IP are there?
len(sourceIP.IP.value_counts())

562248

### Count the frequency for each IP address and sort it by desending order. List the top 10 most frequent IP address.

In [5]:
counts_df = pd.DataFrame(sourceIP.groupby('IP').size().rename('counts'))
counts_list = counts_df.sort(['counts'], ascending=0)
counts_list[:10]
# counts_list = counts_list[1:]


/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,counts
IP,
115.177.11.215,294690
115.176.182.196,259241
192.3.106.42,66572
191.96.249.189,51251
104.192.0.20,18707
198.204.234.26,6817
198.204.234.27,5284
198.204.234.28,5239
185.93.185.10,5157


### Now, let's implement FREQUENT algorithm.

In [6]:
import operator

def frequent(n,k):
    T = {}
    for i in n:
        if i in T:
            T[i] = T[i]+1

        elif len(T) < k:
                T[i]=1
        else:
            for j in T:
                T[j] = T[j]-1          
            new = { k:v for k, v in T.items() if v!= 0 }
#             print(i,new)
            del T
            T = new

    sorted_T = sorted(T.items(), key=operator.itemgetter(1))
    sorted_T.reverse()
#     print(sorted_T.dtypes)
#     sorted_T[0]=counts_list[0]
#     print(sorted_T[:10])
    
#     sorted_10[1] <- count_list[match(as.character(unlist(sorted_10[3:12])),as.character(count_list[[1]])), 2]
#     sorted_10[1] <- with( df, ifelse( sorted_10[1] == "a", X2, X1 )
#     for i in sorted_10[0]:
#         sorted_10[i][1] = counts_list[]

    return sorted_T[:10]


### We use the source IP as input, and iterate through reservoirs of size 10, 100, and 1000. List the top 10 frequent IP addresses and their frequecies.

In [7]:
import numpy as np
n = np.array(sourceIP['IP'])
counter = [10,100,1000]
results = {}

for k in counter:
     results[k]=(frequent(n,k))

In [8]:
## This the top 10 IP and their counts for different K (10,100,1000)
results

{10: [('115.176.182.196', 79417),
  ('191.96.249.189', 696),
  ('187.199.79.13', 2),
  ('87.96.167.236', 1),
  ('187.23.203.254', 1),
  ('114.32.130.19', 1),
  ('82.178.61.97', 1),
  ('115.108.40.16', 1)],
 100: [('115.177.11.215', 266753),
  ('115.176.182.196', 242841),
  ('192.3.106.42', 44058),
  ('191.96.249.189', 37924),
  ('185.93.185.10', 710),
  ('187.199.79.13', 2),
  ('2.27.193.109', 2),
  ('190.66.183.47', 2),
  ('86.105.50.213', 2),
  ('93.100.124.249', 1)],
 1000: [('115.177.11.215', 291988),
  ('115.176.182.196', 257619),
  ('192.3.106.42', 64121),
  ('191.96.249.189', 49629),
  ('104.192.0.20', 16005),
  ('185.93.185.10', 4429),
  ('198.204.234.26', 4219),
  ('198.204.234.27', 2880),
  ('198.204.234.28', 2835),
  ('198.204.234.30', 2645)]}

### Now let's see the actual frequencies for top 10 IP by FREQUENT with different K

In [15]:
#list the top 10 and theri actual frequencies: 


def top10fre(results,k):
    
    IPlist=[]
    top10fre = []
    print("This is the top 10 IP and their actual frequencies: for k = {}".format(k))
    for i in range(len(results[k])):
        IPlist.append(results[k][i][0])
        top10fre.append(counts_list.loc[counts_list['counts'].index == IPlist[i]].counts)
    print([(top10fre[i].index[0],top10fre[i].values[0]) for i in range(0,len(top10fre))])

top10fre(results,10)

This is the top 10 IP and their actual frequencies: for k = 10
[('115.176.182.196', 259241), ('191.96.249.189', 51251), ('187.199.79.13', 84), ('87.96.167.236', 86), ('187.23.203.254', 2), ('114.32.130.19', 1), ('82.178.61.97', 9), ('115.108.40.16', 63)]


In [16]:
top10fre(results,100)

This is the top 10 IP and their actual frequencies: for k = 100
[('115.177.11.215', 294690), ('115.176.182.196', 259241), ('192.3.106.42', 66572), ('191.96.249.189', 51251), ('185.93.185.10', 5157), ('187.199.79.13', 84), ('2.27.193.109', 948), ('190.66.183.47', 84), ('86.105.50.213', 91), ('93.100.124.249', 38)]


In [17]:
top10fre(results,1000)

This is the top 10 IP and their actual frequencies: for k = 1000
[('115.177.11.215', 294690), ('115.176.182.196', 259241), ('192.3.106.42', 66572), ('191.96.249.189', 51251), ('104.192.0.20', 18707), ('185.93.185.10', 5157), ('198.204.234.26', 6817), ('198.204.234.27', 5284), ('198.204.234.28', 5239), ('198.204.234.30', 5049)]
